## The anndata package

### Coding lecture

#### What is anndata?

anndata stands for "Annotated Data". It is a Python package that allows to represent gene expression values, sample/gene metadata, and even results from a single-cell RNA-seq experiment within a single Python object. The main advantage of this approach is a uniform, coherent representaton of all different types of information. Let's see what this means in practice!

In [1]:
# importing anndata
import anndata as ad

# importing numpy and pandas as well
import numpy as np
import pandas as pd

We will now import a subset of the single cell data from the T-cell use case. The data are stored in a compressed format that is optimized for the scRNA-seq experiments.

In [2]:
# importing scRNA-seq data from a .h5ad file
scdr = ad.read_h5ad('py-data/scdr.h5ad')
scdr

AnnData object with n_obs × n_vars = 5016 × 20953
    obs: 'cell.type', 'cytokine.condition', 'donor.id', 'batch.10X', 'nGene', 'nUMI', 'percent.mito', 'S.Score', 'G2M.Score', 'Phase', 'cluster.id', 'effectorness'

Let's dissect this first batch of information. The standard printout of the anndata object shows that the transcriptomics profiles of 5016 cells (denoted as "observations", `obs`) are present. Each profile is composed by 20953 gene expression values, or variables in anndata's terminology. We also have the metadata data frame `obs`, which contains several colums: 'cell.type', 'cytokine.condition', 'donor.id', 'batch.10X', 'nGene', etc. 

Let's first check the gene expression data that are contained in the `X` sparse matrix:

In [3]:
# matrix containing the gene expression values
scdr.X

<5016x20953 sparse matrix of type '<class 'numpy.float32'>'
	with 5709539 stored elements in Compressed Sparse Column format>

Not much to see here, at a first glance at least. The key point is that expression values in single cell experiments are very sparse, i.e., most of the genes have no expression in most of the cells. In these cases it is much more efficient to store only the values that are different from zeros (approx. 5 millions values in our case),rather than the whole matrix with 5016 x 20953 = 105100248 values. 

Let's check what are the highest levels of expressions:

In [5]:
# first 10 highest expression values. 
np.flip(np.sort(scdr.X.data))[range(10)]

array([1130.,  981.,  955.,  911.,  887.,  883.,  859.,  832.,  830.,
        810.], dtype=float32)

We see that the highest expression values across all genes and genes are between 800 / 1100 reads.

Let's now check the `obs` data frame

In [6]:
# obs data frame
scdr.obs

,cell.type,cytokine.condition,donor.id,batch.10X,nGene,nUMI,percent.mito,S.Score,G2M.Score,Phase,cluster.id,effectorness
N_resting_AAACCTGAGCTGTCTA,Naive,UNS,D4,2,1163,4172,0.023496,-0.134199,-0.159211,G1,TN (resting),0.151812
N_resting_AAACCTGTCACCACCT,Naive,UNS,D4,2,1037,3690,0.020867,-0.101756,-0.203707,G1,TN (resting),0.031763
N_resting_AAACCTGTCCGTTGTC,Naive,UNS,D2,2,1245,4446,0.027903,-0.145131,-0.164210,G1,TN (resting),0.113897
N_resting_AAACGGGAGGGTTCCC,Naive,UNS,D4,2,1016,3913,0.011509,-0.069492,-0.190810,G1,TN (resting),0.341240
N_resting_AAACGGGCAACAACCT,Naive,UNS,D1,2,1005,3557,0.039640,-0.124007,-0.143379,G1,TN (resting),0.019741
...,...,...,...,...,...,...,...,...,...,...,...,...
M_resting_r2_TGCTGCTCAATGTAAG,Memory,UNS,D4,2,1235,2936,0.031357,-0.160865,-0.179502,G1,TCM (resting),0.729563
M_resting_r2_CAGCCGATCAGTTTGG,Memory,UNS,D3,2,1128,3690,0.027672,-0.064889,-0.172590,G1,TN (resting),0.255955
M_resting_r1_AGAGCTTCATCTCCCA,Memory,UNS,D2,2,1058,3222,0.038820,-0.078151,-0.206662,G1,TCM (resting),0.478484
M_resting_r2_AAGCCGCCATCGTCGG,Memory,UNS,D2,2,1188,3892,0.036999,-0.155572,-0.139277,G1,TEM (resting),0.813737


The `obs` data frame contains one row for each cell, and each column provides information on a different aspects of the cells. For example, the "cell.type" column allows to distinguish between Naive and Memory T-cells, while "cluster.id" identifies the specific cell type more in detail. Usually, these types of information are not readily available when scRNA-seq data are produced; in this case, they were derived in the original study.

On a side note, the anndata package using pandas for representing tabular data makes a lot of sense, given the versatility of pandas data frames. In general, one advantage of Python and of programming in general is that well-written, useful modules can then be reused as building-block for future packages.

#### Using an anndata object

The information in `obs` can be used for narrowing down the scope of our analysis by subsetting the anndata object. We will now retain only the Naive cells:

In [7]:
# selecting only Naive T-cells
ids = scdr.obs['cell.type'] == 'Naive'
scdr = scdr[ids, :]
scdr

View of AnnData object with n_obs × n_vars = 2141 × 20953
    obs: 'cell.type', 'cytokine.condition', 'donor.id', 'batch.10X', 'nGene', 'nUMI', 'percent.mito', 'S.Score', 'G2M.Score', 'Phase', 'cluster.id', 'effectorness'

We see that `scdr` now contains only the 2141 Naive cells. Let's check the `obs` data frame:

In [8]:
# how many rows on obs are left now?
scdr.obs

,cell.type,cytokine.condition,donor.id,batch.10X,nGene,nUMI,percent.mito,S.Score,G2M.Score,Phase,cluster.id,effectorness
N_resting_AAACCTGAGCTGTCTA,Naive,UNS,D4,2,1163,4172,0.023496,-0.134199,-0.159211,G1,TN (resting),0.151812
N_resting_AAACCTGTCACCACCT,Naive,UNS,D4,2,1037,3690,0.020867,-0.101756,-0.203707,G1,TN (resting),0.031763
N_resting_AAACCTGTCCGTTGTC,Naive,UNS,D2,2,1245,4446,0.027903,-0.145131,-0.164210,G1,TN (resting),0.113897
N_resting_AAACGGGAGGGTTCCC,Naive,UNS,D4,2,1016,3913,0.011509,-0.069492,-0.190810,G1,TN (resting),0.341240
N_resting_AAACGGGCAACAACCT,Naive,UNS,D1,2,1005,3557,0.039640,-0.124007,-0.143379,G1,TN (resting),0.019741
...,...,...,...,...,...,...,...,...,...,...,...,...
N_resting_TTTGCGCAGGAGTCTG,Naive,UNS,D1,2,980,3233,0.046720,-0.048484,-0.172172,G1,TEMRA (resting),0.837294
N_resting_TTTGTCAAGGATCGCA,Naive,UNS,D1,2,754,2016,0.051091,-0.122710,-0.161167,G1,TEMRA (resting),0.903606
N_resting_TTTGTCAGTCCGTCAG,Naive,UNS,D1,2,964,2725,0.037826,-0.109837,-0.196064,G1,TEMRA (resting),0.914887
N_resting_TTTGTCAGTGCTTCTC,Naive,UNS,D1,2,1027,3003,0.052316,-0.144304,-0.225455,G1,TEMRA (resting),0.869464


Here an important point: *subsetting the anndata object with the command `scdr = scdr[ids, :]` has modified both the gene expression matrix X and the metadata data frame `obs`*. This is the essence of having a single, unified object for representing all information produced in a scRNA-seq experiments: changes are automatically propagated through all the relevant data and metadata in a coherent way.

Let's now add another type of metadata to the `scdr` object: the `var` data frame that contains information on each single gene. For this, we will first compute the total number of reads for each gene across all cells

In [9]:
# summing over the first axis / dimension
num_reads = scdr.X.sum(axis=0)
num_reads

matrix([[  3., 263.,  97., ...,   0.,   0.,   0.]], dtype=float32)

Let's now create the `var` data frame:

In [10]:
# creating the vars data frame
scdr.var = pd.DataFrame({'num_reads':np.array(num_reads).flatten()}, index=scdr.var_names)
scdr.var

,num_reads
RP11-34P13.7,3.0
FO538757.2,263.0
AP006222.2,97.0
RP4-669L17.10,9.0
RP11-206L10.9,39.0
...,...
MEF2B,0.0
AC002398.12,0.0
AC005625.1,0.0
AC115522.3,0.0


We now have one more metadata information, this time focusing on characterizing genes, not cells. This change is of course also reflected in the anndata object standard printout, with `var` appearing right below `obs`.

In [11]:
# showing scdr content
scdr

AnnData object with n_obs × n_vars = 2141 × 20953
    obs: 'cell.type', 'cytokine.condition', 'donor.id', 'batch.10X', 'nGene', 'nUMI', 'percent.mito', 'S.Score', 'G2M.Score', 'Phase', 'cluster.id', 'effectorness'
    var: 'num_reads'

Many more types of information can be stored within an anndata object. We will see most of them during the next coding lessons, when we will start preprocessing the data.

#### Reading / writing anndata objects

We already used the `read_h5ad` function for reading an anndata object saved in a binary file based on the hdf5 format. You can also use the `write_h5ad` function for creating a similar file for our current `scdr` object:

In [12]:
# let's import the os module for managing files and directories
import os

# creating a directory for storing the new data
if not os.path.exists('output_data'): 
    os.makedirs('output_data')

# writing h5ad objects
scdr.write_h5ad('output_data/scdr_naive_cells.h5ad')

# check the content of the directory
os.listdir('output_data')

['scdr_naive_cells.h5ad']

The hdf5 format is unparalled in terms of reading / writing speed, especially for very large files. However, if you want to store your anndata object in comma separate values (csv) files, you can use the `write_csvs` function:

In [13]:
# using the write_csv function:
scdr.write_csvs('output_data/csvs', skip_data=True)

# listing all written files
os.listdir('output_data/csvs')

['obs.csv', 'obsm.csv', 'var.csv', 'varm.csv', 'uns']

This function has created a new folder, "csvs", which contain one csv file for each element contained within the `scdr` object. *The X matrix is not written, though, unless you explicitly set the `skip_data` argument to `False`*. This is because writing the gene expression matrix in a csv format would usually take very large amount of disk space. In our case `X` would take ~180 MB in csv format, but larger experiments would easily scale up to several GB.